# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
# YOUR CHANGES HERE
import yfinance as yf
import cvxpy as cp
import pandas as pd

tickers = ["AAPL", "SPY", "TSLA", "YETI", "MSFT", "NVDA"]

historical_prices = None

for t in tickers:
    ticker = yf.Ticker(t)
    history = ticker.history(period="2y")
    
    monthly_prices = history['Close'].resample('ME').last()
    
    if historical_prices is None:
        historical_prices = monthly_prices
    else:
        historical_prices = pd.concat([historical_prices, monthly_prices], axis=1)

historical_prices.columns = tickers
historical_prices.index = pd.to_datetime(historical_prices.index).strftime("%Y-%m-%d")
historical_prices = historical_prices.tail(24)

historical_prices = historical_prices.reset_index()
historical_prices = historical_prices.rename(columns={'index': 'Date'})  
historical_prices = historical_prices.rename(columns={'Date': 'date'}) 


print("Fixed columns:", historical_prices.columns.tolist())
print(historical_prices.head())

Fixed columns: ['date', 'AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
         date        AAPL         SPY        TSLA       YETI        MSFT  \
0  2023-10-31  169.113556  408.794403  200.839996  42.520000  333.090332   
1  2023-11-30  188.355331  446.135193  240.080002  42.639999  374.042267   
2  2023-12-31  190.913666  466.503693  248.479996  51.779999  371.209137   
3  2024-01-31  182.851929  473.933441  187.289993  43.970001  392.472412   
4  2024-02-29  179.461243  498.666504  201.880005  41.040001  409.081238   

        NVDA  
0  40.758278  
1  46.745087  
2  49.499973  
3  61.499630  
4  79.076813  


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [2]:
# YOUR CHANGES HERE
historical_prices.to_csv('historical_prices.tsv', sep='\t', index=False)
print("\nData saved to 'historical_prices.tsv'")


Data saved to 'historical_prices.tsv'


Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [3]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

historical_prices = pd.read_csv('historical_prices.tsv', sep='\t')

print("Columns in your data:", historical_prices.columns.tolist())
print("First few rows:")
print(historical_prices.head())

numeric_columns = historical_prices.select_dtypes(include=[np.number]).columns
print(f"\nNumeric columns found: {numeric_columns.tolist()}")

historical_returns = historical_prices[numeric_columns].pct_change().dropna()

print(f"\nHistorical returns calculated:")
print(f"Shape: {historical_returns.shape}")
print(historical_returns.head())

dates_for_returns = historical_prices['date'].iloc[1:].reset_index(drop=True)

stock_columns = ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
returns_only = historical_returns[stock_columns].reset_index(drop=True)

returns_data = pd.concat([
    dates_for_returns.rename('date'),
    returns_only
], axis=1)

print("\nFinal returns data with date column:")
print(returns_data.head())
print(f"Shape: {returns_data.shape}")
print(f"Columns: {returns_data.columns.tolist()}")

Columns in your data: ['date', 'AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
First few rows:
         date        AAPL         SPY        TSLA       YETI        MSFT  \
0  2023-10-31  169.113556  408.794403  200.839996  42.520000  333.090332   
1  2023-11-30  188.355331  446.135193  240.080002  42.639999  374.042267   
2  2023-12-31  190.913666  466.503693  248.479996  51.779999  371.209137   
3  2024-01-31  182.851929  473.933441  187.289993  43.970001  392.472412   
4  2024-02-29  179.461243  498.666504  201.880005  41.040001  409.081238   

        NVDA  
0  40.758278  
1  46.745087  
2  49.499973  
3  61.499630  
4  79.076813  

Numeric columns found: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']

Historical returns calculated:
Shape: (23, 6)
       AAPL       SPY      TSLA      YETI      MSFT      NVDA
1  0.113780  0.091344  0.195379  0.002822  0.122945  0.146886
2  0.013582  0.045655  0.034988  0.214353 -0.007574  0.058934
3 -0.042227  0.015926 -0.246257 -0.150830  0.057281  0

Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [4]:
# YOUR CHANGES HERE

returns_data.to_csv('historical_returns.tsv', sep='\t', index=False)
print("Historical returns saved to 'historical_returns.tsv'")

Historical returns saved to 'historical_returns.tsv'


Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [5]:
# YOUR CHANGES HERE

import pandas as pd

historical_returns_df = pd.read_csv('historical_returns.tsv', sep='\t')

print("Historical returns data loaded:")
print(f"Shape: {historical_returns_df.shape}")
print(historical_returns_df.head())

stock_columns = [col for col in historical_returns_df.columns if col != 'date']
print(f"\nStock columns: {stock_columns}")

mean_returns = historical_returns_df[stock_columns].mean()

print("\nCalculated mean returns:")
print(mean_returns)

original_order = ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
estimated_returns_df = pd.DataFrame({
    'asset': mean_returns.index,
    'estimated_return': mean_returns.values
})

print(estimated_returns_df)

print("\nInterpretation:")
for _, row in estimated_returns_df.iterrows():
    asset = row['asset']
    return_val = row['estimated_return']
    annual_return = return_val * 12  # Convert monthly to approximate annual
    print(f"{asset}: {return_val:.6f} monthly ({annual_return:.2%} annualized)")

Historical returns data loaded:
Shape: (23, 7)
         date      AAPL       SPY      TSLA      YETI      MSFT      NVDA
0  2023-11-30  0.113780  0.091344  0.195379  0.002822  0.122945  0.146886
1  2023-12-31  0.013582  0.045655  0.034988  0.214353 -0.007574  0.058934
2  2024-01-31 -0.042227  0.015926 -0.246257 -0.150830  0.057281  0.242417
3  2024-02-29 -0.018543  0.052187  0.077901 -0.066636  0.042318  0.285810
4  2024-03-31 -0.051286  0.032702 -0.129235 -0.060673  0.017116  0.142181

Stock columns: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']

Calculated mean returns:
AAPL    0.016968
SPY     0.020758
TSLA    0.036103
YETI   -0.002163
MSFT    0.019236
NVDA    0.069567
dtype: float64
  asset  estimated_return
0  AAPL          0.016968
1   SPY          0.020758
2  TSLA          0.036103
3  YETI         -0.002163
4  MSFT          0.019236
5  NVDA          0.069567

Interpretation:
AAPL: 0.016968 monthly (20.36% annualized)
SPY: 0.020758 monthly (24.91% annualized)
TSLA: 0.036103 mon

Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [6]:
# YOUR CHANGES HERE
estimated_returns_df.to_csv('estimated_returns.tsv', sep='\t', index=False)
print("Estimated returns saved to 'estimated_returns.tsv'")
    

Estimated returns saved to 'estimated_returns.tsv'


Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [7]:
# YOUR CHANGES HERE

import pandas as pd

historical_returns_df = pd.read_csv('historical_returns.tsv', sep='\t')

stock_columns = [col for col in historical_returns_df.columns if col != 'date']

returns_data = historical_returns_df[stock_columns]
covariance_matrix = returns_data.cov()

print("Original covariance matrix:")
print(covariance_matrix)

covariance_matrix.index.name = ''  

print(f"\nFixed covariance matrix (with blank index name):")
print(covariance_matrix)

print(f"\nVerifying file format:")
with open('estimated_covariance.tsv', 'r') as f:
    lines = f.readlines()
    print(f"Header row: '{lines[0].strip()}'")
    print(f"First data row: '{lines[1].strip()}'")

Original covariance matrix:
          AAPL       SPY      TSLA      YETI      MSFT      NVDA
AAPL  0.003623  0.000784  0.003075  0.002191  0.000398  0.000768
SPY   0.000784  0.001225  0.001971  0.001676  0.001616  0.002673
TSLA  0.003075  0.001971  0.024358  0.005674  0.002690 -0.001848
YETI  0.002191  0.001676  0.005674  0.010036  0.001532  0.001264
MSFT  0.000398  0.001616  0.002690  0.001532  0.004022  0.005432
NVDA  0.000768  0.002673 -0.001848  0.001264  0.005432  0.014287

Fixed covariance matrix (with blank index name):
          AAPL       SPY      TSLA      YETI      MSFT      NVDA
                                                                
AAPL  0.003623  0.000784  0.003075  0.002191  0.000398  0.000768
SPY   0.000784  0.001225  0.001971  0.001676  0.001616  0.002673
TSLA  0.003075  0.001971  0.024358  0.005674  0.002690 -0.001848
YETI  0.002191  0.001676  0.005674  0.010036  0.001532  0.001264
MSFT  0.000398  0.001616  0.002690  0.001532  0.004022  0.005432
NVDA  0.0007

Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [8]:
# YOUR CHANGES HERE

covariance_matrix.to_csv('estimated_covariance.tsv', sep='\t')
print("Covariance matrix saved to 'estimated_covariance.tsv'")

Covariance matrix saved to 'estimated_covariance.tsv'


Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [9]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

estimated_returns_df = pd.read_csv('estimated_returns.tsv', sep='\t')

print("Estimated returns data:")
print(estimated_returns_df)

max_return_idx = estimated_returns_df['estimated_return'].idxmax()
best_asset = estimated_returns_df.loc[max_return_idx, 'asset']
max_return_value = estimated_returns_df.loc[max_return_idx, 'estimated_return']

print(f"\nAsset with highest expected return:")
print(f"Asset: {best_asset}")
print(f"Expected return: {max_return_value:.6f}")

all_assets = estimated_returns_df['asset'].tolist()
allocations = []

for asset in all_assets:
    if asset == best_asset:
        allocation = 1.0  
    else:
        allocation = 0.0  
    allocations.append(allocation)

max_return_portfolio = pd.DataFrame({
    'asset': all_assets,
    'allocation': allocations
})

print(f"\nMaximum Return Portfolio:")
print(max_return_portfolio)

total_allocation = max_return_portfolio['allocation'].sum()
print(f"\nTotal allocation: {total_allocation:.6f} (should be 1.0)")

portfolio_return = (max_return_portfolio['allocation'] * 
                   estimated_returns_df['estimated_return']).sum()
print(f"Portfolio expected return: {portfolio_return:.6f}")

Estimated returns data:
  asset  estimated_return
0  AAPL          0.016968
1   SPY          0.020758
2  TSLA          0.036103
3  YETI         -0.002163
4  MSFT          0.019236
5  NVDA          0.069567

Asset with highest expected return:
Asset: NVDA
Expected return: 0.069567

Maximum Return Portfolio:
  asset  allocation
0  AAPL         0.0
1   SPY         0.0
2  TSLA         0.0
3  YETI         0.0
4  MSFT         0.0
5  NVDA         1.0

Total allocation: 1.000000 (should be 1.0)
Portfolio expected return: 0.069567


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [10]:
# YOUR CHANGES HERE

max_return_portfolio.to_csv('maximum_return.tsv', sep='\t', index=False, float_format='%.6f')
print("Maximum return portfolio saved to 'maximum_return.tsv'")

Maximum return portfolio saved to 'maximum_return.tsv'


Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [51]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
import cvxpy as cp

estimated_returns_df = pd.read_csv("estimated_returns.tsv", sep="\t")
estimated_covariance = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)

assets = estimated_returns_df['asset'].tolist()
estimated_covariance = estimated_covariance.reindex(index=assets, columns=assets)
estimated_returns = estimated_returns_df.set_index('asset')['estimated_return'].reindex(assets)

n = len(estimated_returns)
x_minimum_risk = cp.Variable(n)

objective_minimum_risk = cp.Minimize(x_minimum_risk.T @ estimated_covariance.to_numpy() @ x_minimum_risk)

prob_minimum_risk = cp.Problem(objective_minimum_risk, 
                              [0 <= x_minimum_risk, 
                               cp.sum(x_minimum_risk) == 1])

covariance_minimum_risk = prob_minimum_risk.solve()

if prob_minimum_risk.status != cp.OPTIMAL:
    raise RuntimeError(f"CVXPy optimization failed: {prob_minimum_risk.status}")

min_risk_weights = x_minimum_risk.value

print("Minimum risk portfolio calculated using CVXPy")
print(f"Optimization status: {prob_minimum_risk.status}")
print(f"Weights sum: {min_risk_weights.sum()}")


Minimum risk portfolio calculated using CVXPy
Optimization status: optimal
Weights sum: 1.0


Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [52]:
# YOUR CHANGES HERE

min_risk_df = pd.DataFrame({
    "asset": assets,
    "allocation": min_risk_weights
})

min_risk_df.to_csv("minimum_risk.tsv", sep="\t", index=False)
print("Minimum risk portfolio saved to minimum_risk.tsv")
print(min_risk_df)


Minimum risk portfolio saved to minimum_risk.tsv
  asset    allocation
0  AAPL  1.345446e-01
1   SPY  8.654554e-01
2  TSLA -1.957845e-21
3  YETI -1.414751e-20
4  MSFT -4.997472e-20
5  NVDA -6.419999e-20


Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [ ]:
# YOUR CHANGES HERE

import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.optimize import minimize

est = pd.read_csv("estimated_returns.tsv", sep="\t")
estimated_covariance = pd.read_csv("estimated_covariance.tsv", sep="\t", index_col=0)

assets = est['asset'].tolist()
estimated_covariance = estimated_covariance.reindex(index=assets, columns=assets)
mu_vec = est.set_index('asset')['estimated_return'].reindex(assets).values
Sigma = estimated_covariance.to_numpy()
n = len(assets)

x_minimum_risk = cp.Variable(n)
objective_minimum_risk = cp.Minimize(x_minimum_risk.T @ Sigma @ x_minimum_risk)
prob_minimum_risk = cp.Problem(objective_minimum_risk, [0 <= x_minimum_risk, cp.sum(x_minimum_risk) == 1])
prob_minimum_risk.solve()
w_gmv = x_minimum_risk.value
r_gmv = float(w_gmv @ mu_vec)

best_asset_idx = np.argmax(mu_vec)
w_max = np.zeros(n)
w_max[best_asset_idx] = 1.0
r_max = float(w_max @ mu_vec)

targets = np.linspace(r_gmv, r_max, 101)

def min_var_for_target(Sigma, mu_vec, target):
    x = cp.Variable(n)
    objective = cp.Minimize(x.T @ Sigma @ x)
    constraints = [0 <= x, cp.sum(x) == 1, x.T @ mu_vec == target]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return prob, x.value

records = []

for i, tr in enumerate(targets):
    if i == 0:
        w_sol = w_gmv
        r_port = r_gmv
    elif i == 100:
        w_sol = w_max
        r_port = r_max
    else:
        prob, w_sol = min_var_for_target(Sigma, mu_vec, tr)
        if w_sol is None:
            alpha = i / 100.0
            w_sol = (1 - alpha) * w_gmv + alpha * w_max
        r_port = float(w_sol @ mu_vec)
    
    vol_port = float(np.sqrt(w_sol @ Sigma @ w_sol))
    record = [i, r_port, vol_port] + list(w_sol)
    records.append(record)
columns = ["index", "return", "risk"] + assets
frontier = pd.DataFrame(records, columns=columns)

Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [56]:
# YOUR CHANGES HERE

frontier.to_csv("efficient_frontier.tsv", sep="\t", index=False)
print("Efficient frontier saved to 'efficient_frontier.tsv'")

Efficient frontier saved to 'efficient_frontier.tsv'


Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [37]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

covariance_matrix = pd.read_csv('estimated_covariance.tsv', sep='\t', index_col=0)
estimated_returns_df = pd.read_csv('estimated_returns.tsv', sep='\t')

print("Data loaded for stability check")

assets = list(covariance_matrix.index)
n_assets = len(assets)
cov_matrix = covariance_matrix.values

expected_returns = np.zeros(n_assets)
for i, asset in enumerate(assets):
    expected_returns[i] = estimated_returns_df[estimated_returns_df['asset'] == asset]['estimated_return'].iloc[0]

print(f"Assets: {assets}")
print(f"Original expected returns: {expected_returns}")

original_max_asset = estimated_returns_df.loc[estimated_returns_df['estimated_return'].idxmax(), 'asset']
print(f"Original maximum return asset: {original_max_asset}")

n_simulations = 1000
n_samples = 23  

print(f"\nRunning {n_simulations} Monte Carlo simulations...")
print(f"Each simulation generates {n_samples} return samples")

highest_return_assets = []

np.random.seed(42)

for sim in range(n_simulations):
    simulated_returns = np.random.multivariate_normal(
        mean=expected_returns, 
        cov=cov_matrix, 
        size=n_samples
    )
    
    sim_mean_returns = np.mean(simulated_returns, axis=0)
    
    max_return_idx = np.argmax(sim_mean_returns)
    highest_return_asset = assets[max_return_idx]
    highest_return_assets.append(highest_return_asset)
   
    if (sim + 1) % 200 == 0:
        print(f"Completed {sim + 1} simulations...")

print(f"\nSimulation completed!")

from collections import Counter
asset_counts = Counter(highest_return_assets)

print(f"\nResults summary:")
print(f"Asset with highest return in each simulation:")
for asset in assets:
    count = asset_counts.get(asset, 0)
    probability = count / n_simulations
    print(f"{asset}: {count} times ({probability:.3f} probability)")

results_df = pd.DataFrame({
    'asset': assets,
    'probability': [asset_counts.get(asset, 0) / n_simulations for asset in assets]
})

print(f"\nStability analysis:")
original_prob = results_df[results_df['asset'] == original_max_asset]['probability'].iloc[0]
print(f"Original choice ({original_max_asset}) was optimal in {original_prob:.1%} of simulations")

if original_prob < 0.5:
    print(f"Warning: Original choice was not optimal in majority of simulations")
    print(f"This suggests the maximum return portfolio choice is not very stable")
else:
    print(f" Original choice was optimal in majority of simulations")
    print(f"This suggests the maximum return portfolio choice is relatively stable")
   
results_sorted = results_df.sort_values('probability', ascending=False)
print(f"\nAssets ranked by probability of having highest return:")
print(results_sorted)

Data loaded for stability check
Assets: ['AAPL', 'SPY', 'TSLA', 'YETI', 'MSFT', 'NVDA']
Original expected returns: [ 0.01696782  0.02075796  0.0361031  -0.00216279  0.0192359   0.06956662]
Original maximum return asset: NVDA

Running 1000 Monte Carlo simulations...
Each simulation generates 23 return samples
Completed 200 simulations...
Completed 400 simulations...
Completed 600 simulations...
Completed 800 simulations...
Completed 1000 simulations...

Simulation completed!

Results summary:
Asset with highest return in each simulation:
AAPL: 3 times (0.003 probability)
SPY: 0 times (0.000 probability)
TSLA: 207 times (0.207 probability)
YETI: 1 times (0.001 probability)
MSFT: 0 times (0.000 probability)
NVDA: 789 times (0.789 probability)

Stability analysis:
Original choice (NVDA) was optimal in 78.9% of simulations
 Original choice was optimal in majority of simulations
This suggests the maximum return portfolio choice is relatively stable

Assets ranked by probability of having hig

Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [38]:
# YOUR CHANGES HERE

results_df.to_csv('max_return_probabilities.tsv', sep='\t', index=False)
print("Maximum return portfolio stability results saved to 'max_return_probabilities.tsv'")

Maximum return portfolio stability results saved to 'max_return_probabilities.tsv'


Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.